In [1]:
import torch.nn.functional as F

In [12]:
weighted_classes = [1] * 29
weighted_classes = [19, 20, 25, 26]

In [2]:
import torch

In [3]:
import torch.nn as nn
loss = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(3))

In [48]:
CLASSES = [
    'finger-1', 'finger-2', 'finger-3', 'finger-4', 'finger-5',
    'finger-6', 'finger-7', 'finger-8', 'finger-9', 'finger-10',
    'finger-11', 'finger-12', 'finger-13', 'finger-14', 'finger-15',
    'finger-16', 'finger-17', 'finger-18', 'finger-19', 'Trapezium',
    'Trapezoid', 'Capitate', 'Hamate', 'Scaphoid', 'Lunate',
    'Triquetrum', 'Pisiform', 'Radius', 'Ulna',
]

CLASS2IND = {v: i for i, v in enumerate(CLASSES)}

class DiceLoss:
    def __init__(self, smooth=1.0, sigmoid=True, class_weight=None):
        self.smooth = smooth
        self.sigmoid = sigmoid
        self.class_weight = torch.ones([29])
        if class_weight is not None:
            for c, w in class_weight.items():
                self.class_weight[CLASS2IND[c]] = w

    def __call__(self, pred, target):
        if self.sigmoid:
            pred = F.sigmoid(pred)    
        pred = pred.contiguous()
        target = target.contiguous()
        intersection = (pred * target).sum(dim=2).sum(dim=2)
        loss = 1 - (
            (2.0 * intersection + self.smooth)
            / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + self.smooth)
        )
        print
        loss.mul_(self.class_weight)
        return loss.mean()


class BCEDiceLoss:
    def __init__(self, bce_weight=0.5, class_weight=None):
        self.bce_weight = bce_weight
        self.class_weight = torch.ones([29])
        if class_weight is not None:
            for c, w in class_weight.items():
                self.class_weight[CLASS2IND[c]] = w
        self.dice_loss = DiceLoss(sigmoid=False, class_weight=class_weight)

    def __call__(self, pred, target):
        pred_flatten = pred.view(pred.size(0), -1, pred.size(1))
        target_flatten = target.view(target.size(0), -1, target.size(1))
        bce = F.binary_cross_entropy_with_logits(pred_flatten, target_flatten,weight=self.class_weight, pos_weight=self.class_weight)
        pred = F.sigmoid(pred)
        dice = self.dice_loss(pred, target)
        loss = bce * self.bce_weight + dice * (1 - self.bce_weight)
        return loss


In [2]:
x = torch.ones((29))

NameError: name 'torch' is not defined

In [45]:
import torch
pred = torch.rand((2, 29, 512, 512))
label = torch.rand((2, 29, 512, 512))


In [49]:
((label[0, 0, :]==0.).sum()/label[0, 0, :].sum())

tensor(0.)

In [49]:
loss = BCEDiceLoss(class_weight={"Trapezoid":1.5, "Pisiform":2})
print(loss(pred, label))

tensor(0.6321)


In [33]:
a = torch.tensor([[1,1,1,1]])
a.mul_(torch.tensor([1,2,1,2]))
print(a)

tensor([[1, 2, 1, 2]])


In [28]:
torch.tensor([1,2,1,2]).shape

torch.Size([4])